In [0]:
%fs

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
dbutils.fs.mount(source='wasbs://olympicdata@tokyoolympicdatajeswanth.blob.core.windows.net',mount_point='/mnt/olympicdata',
extra_configs={'fs.azure.account.key.tokyoolympicdatajeswanth.blob.core.windows.net':'ULhSOyD536VBqMsW46aDm5XNEfWk5WHVRBUvzzPilnh7a9eh7Led9aMQx956HyXHSKZkoBm8zLGD+ASt5WrxtQ=='})

True

dbutils.fs.mount(source='wasbs://<containername>@<storage_accout_name>.blob.core.windows.net',mount_point='/mnt/olympicdata',
extra_configs={'fs.azure.account.key.<storage_accout_name>.blob.core.windows.net':'ULhSOyD536VBqMsW46aDm5XNEfWk5WHVRBUvzzPilnh7a9eh7Led9aMQx956HyXHSKZkoBm8zLGD+ASt5WrxtQ=='})


In [0]:
spark

In [0]:
athe_df=spark.read.format("csv").option("header",True).load("dbfs:/mnt/olympicdata/raw_data/atheletes.csv")
coach_df=spark.read.format("csv").option("header",True).load("dbfs:/mnt/olympicdata/raw_data/coaches.csv")
gender_df=spark.read.format("csv").option("header",True).load("dbfs:/mnt/olympicdata/raw_data/Gender.csv")
medals_df=spark.read.format("csv").option("header",True).load("dbfs:/mnt/olympicdata/raw_data/medals.csv")
teams_df=spark.read.format("csv").option("header",True).load("dbfs:/mnt/olympicdata/raw_data/teams.csv")


In [0]:
athe_df.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
coach_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
gender_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
medals_df.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
teams_df.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
gender_new_df=gender_df.withColumn("Female",col("Female").cast(IntegerType())) \
        .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))
                                                                        

In [0]:
gender_new_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
medals_df.show(2)

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
+----+--------------------+----+------+------+-----+-------------+
only showing top 2 rows



In [0]:
#find the country with highest gold medals
top_gold_medals_df = medals_df.orderBy("Gold", ascending=False).select("Team_Country", "Gold").show(20)


+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|                Cuba|   7|
|         New Zealand|   7|
|              Canada|   7|
|              Brazil|   7|
|   Republic of Korea|   6|
|             Hungary|   6|
|              Poland|   4|
|             Jamaica|   4|
|               Kenya|   4|
|      Czech Republic|   4|
|              Norway|   4|
|United States of ...|  39|
|People's Republic...|  38|
|              Sweden|   3|
|               Spain|   3|
|             Denmark|   3|
|         Switzerland|   3|
|             Croatia|   3|
|              Serbia|   3|
|Islamic Republic ...|   3|
+--------------------+----+
only showing top 20 rows



In [0]:
athe_df.repartition(3).write.mode("overwrite").option("header",True).format("csv").save("/mnt/olympicdata/transformed_data/atheletes")

In [0]:
coach_df.repartition(3).write.mode("overwrite").option("header",True).format("csv").save("/mnt/olympicdata/transformed_data/coaches")
medals_df.repartition(3).write.mode("overwrite").option("header",True).format("csv").save("/mnt/olympicdata/transformed_data/medals")
teams_df.repartition(3).write.mode("overwrite").option("header",True).format("csv").save("/mnt/olympicdata/transformed_data/teams")
gender_new_df.repartition(3).write.mode("overwrite").option("header",True).format("csv").save("/mnt/olympicdata/transformed_data/gender")